This notebook is organized by the 6-step Data Analysis framework (Ask, Prepare, Processs, Analyze, Recommend, Share) put forth within the Google Data Analytics Certificate Course. 

For a high-level summary of this analysis jump to the final section of this notebook. A summary of the findings from this analysis is also presented in a Tableau story format here.

# PREPARE | Collect Data

The data used for this analysis is a public domain (CC0 1.0 Universal Public Domain) dataset made available on Kaggle (via user MÖBIUS).

Data source: [FitBit Fitness Tracker Data](https://www.kaggle.com/datasets/arashnic/fitbit)  

### Data Details
The data uploaded by Kaggle user MÖBIUS originates from respondents to a distributed survey via Amazon Mechanical Turk from March 12 to May 12, 2016. Thirty-three Fitbit users submitted personal tracker data collected in 18 files covering:
- Physical Activity (measured in Steps, Calories, and METs (metabolic equivalents)
- Sleep (measured in minutes)
- Heart rate (bpm)
- Weight/BMI (lbs/kg)

Data covers a 30-day period (04.12.16 - 05.12.16).



This analysis will focus on **Physical Activity** (daily, hourly), **Sleep** (daily), and **weight/bmi** to understand usage at a broader level. 


While 33 unique individuals provided data for physical activity, the other measured data contained fewer individuals.  
- Physical Activity: 33
- Sleep monitoring: 24
- Weight: 8
- Heart rate: 14 

### Licensing, Privacy, Security, Integrity
[CC0 1.0 Universal Public Domain](https://creativecommons.org/publicdomain/zero/1.0/)

All users participating in the survey consented to the submission of personal tracking data. The users' privacy has been protected by only identifying unique individuals via randomly generated ID numbers. This data has been provided by a 3rd party, the Kaggle user MÖBIUS.

### Data Integrity
- Sample selection bias since, insignificant 
- Variation in output due to different types of Fitbit trackers
- Variation in individual tracking behavior/preferences
- Concerns that there is no demographic data (sex, age, location) 
- Obsolescence - 5 years old



In [1]:
import pandas as pd
import pandas_gbq
import os

from google.cloud import bigquery
%load_ext google.cloud.bigquery

pandas_gbq.context.project = 'gac-bellabeat'
pandas_gbq.context.dialect = 'standard'

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/Users/atat/code/phlln/gcp_keys/gac-bellabeat-jupyter-bigquery-key.json'

In [34]:
%%bigquery
SELECT *
  FROM `gac-bellabeat.activity.sleep`
  LIMIT 1

Downloading: 100%|██████████| 1/1 [00:01<00:00,  1.67s/rows]


,Id,SleepDay,TotalSleepRecords,TotalMinutesAsleep,TotalTimeInBed
0,1503960366,2016-04-12 00:00:00+00:00,1,327,346


## PROCESS | Wrangle Data

Inspect, Wrangle, Validate Data
- Check data type, number of records, number of nulls, stat summary
- addition of columns for easier analysis _by day of week and hour
- We'll look at Physical Activity, Sleep, Weight/BMI

### Data Details
The data uploaded by Kaggle user MÖBIUS originates from respondents to a distributed survey via Amazon Mechanical Turk from March 12 to May 12, 2016. Thirty-three Fitbit users submitted personal tracker data collected in 18 files covering:
- Physical Activity (measured in Steps, Calories, and METs (metabolic equivalents)
- Sleep (measured in minutes)
- Weight/BMI (lbs/kg)
- Heart rate (bpm)

Data covers a 30-day period (04.12.16 - 05.12.16).

This analysis will focus on **Physical Activity** (daily, hourly), **Sleep** (daily), and **weight/bmi** to understand usage at a broader level. 


First, we'll check the columns and data types for the following tables: daily, hourly, sleep, and weight. We'll also exclude columns tracking distance instead preferring those using time or steps. 

In [3]:
%%bigquery
SELECT table_name,
       column_name,
       data_type
  FROM `gac-bellabeat.activity.INFORMATION_SCHEMA.COLUMNS`
WHERE column_name NOT LIKE '%Distance'; 

Downloading: 100%|██████████| 39/39 [00:02<00:00, 19.49rows/s]


,table_name,column_name,data_type
0,daily,Id,INT64
1,daily,ActivityDate,DATE
2,daily,TotalSteps,INT64
3,daily,VeryActiveMinutes,INT64
4,daily,FairlyActiveMinutes,INT64
5,daily,LightlyActiveMinutes,INT64
6,daily,SedentaryMinutes,INT64
7,daily,Calories,INT64
8,sleep,Id,INT64
9,sleep,SleepDay,TIMESTAMP


### Activity Data

The most detailed and complete data is for logging physical activity. Various metrics for measuring physical exertion (Steps, Calories, METs) are provided whether measure by time or distance. Additionally, levels of intensity have been categorized into four levels ('VeryActive', 'FairlyActive', 'LightlyActive', 'Sedentary'). This data has also been recorded at differnt time scales (daily, hourly, and minute).   

#### Daily Activity Data

##### Validate number of unique individuals

In [4]:
%%bigquery
SELECT
    COUNT(DISTINCT ID) AS user_count
FROM `gac-bellabeat.activity.daily`;
  

Downloading: 100%|██████████| 1/1 [00:01<00:00,  1.42s/rows]


,user_count
0,33


##### Check for Null Values

In [5]:
%%bigquery
SELECT
    COUNT(*) total_rows,
    SUM(CASE WHEN ActivityDate IS NULL THEN 1 ELSE 0 END) activitydate_num_null,
    SUM(CASE WHEN TotalSteps IS NULL THEN 1 ELSE 0 END) totalsteps_num_null,
    SUM(CASE WHEN VeryActiveMinutes IS NULL THEN 1 ELSE 0 END) veryactive_num_null,
    SUM(CASE WHEN FairlyActiveMinutes IS NULL THEN 1 ELSE 0 END) fairlyactive_num_null,
    SUM(CASE WHEN LightlyActiveMinutes IS NULL THEN 1 ELSE 0 END) lightlyactive_num_null,
    SUM(CASE WHEN SedentaryMinutes IS NULL THEN 1 ELSE 0 END) sedentary_num_null,
    SUM(CASE WHEN Calories IS NULL THEN 1 ELSE 0 END) calories_num_null
FROM `gac-bellabeat.activity.daily`;

Downloading: 100%|██████████| 1/1 [00:01<00:00,  1.40s/rows]


,total_rows,activitydate_num_null,totalsteps_num_null,veryactive_num_null,fairlyactive_num_null,lightlyactive_num_null,sedentary_num_null,calories_num_null
0,940,0,0,0,0,0,0,0


##### Create summary stats panel (Total, Mean, Min, 25%, 50%, 75%, Max, Std)

In [6]:
%%bigquery
SELECT 1 AS Index, 
      'Count' AS Statistic,
       COUNT(TotalSteps) AS total_steps,
       COUNT(VeryActiveMinutes) AS very_active_minutes,
       COUNT(FairlyActiveMinutes) AS fairly_active_minutes,
       COUNT(LightlyActiveMinutes) AS lightly_active_minutes,
       COUNT(SedentaryMinutes) AS sedentary_minutes
  FROM `gac-bellabeat.activity.daily`
 UNION ALL
SELECT 2, 
      'Mean',
       ROUND(AVG(TotalSteps), 1),
       ROUND(AVG(VeryActiveMinutes), 1),
       ROUND(AVG(FairlyActiveMinutes), 1),
       ROUND(AVG(LightlyActiveMinutes), 1),
       ROUND(AVG(SedentaryMinutes), 1)
  FROM `gac-bellabeat.activity.daily`
 UNION ALL
(SELECT 3,
       'STD',
       ROUND(STDDEV_POP(TotalSteps) OVER(), 1),
       ROUND(STDDEV_POP(VeryActiveMinutes) OVER(), 1),
       ROUND(STDDEV_POP(FairlyActiveMinutes) OVER(), 1),
       ROUND(STDDEV_POP(LightlyActiveMinutes) OVER(), 1),
       ROUND(STDDEV_POP(SedentaryMinutes) OVER(), 1)
  FROM `gac-bellabeat.activity.daily`
 LIMIT 1)
 UNION ALL
SELECT 4,
       'Min',
       MIN(TotalSteps),
       MIN(VeryActiveMinutes),
       MIN(FairlyActiveMinutes),
       MIN(LightlyActiveMinutes),
       MIN(SedentaryMinutes)
  FROM `gac-bellabeat.activity.daily`
  UNION ALL
(SELECT 5,
       '25%',
       PERCENTILE_CONT(TotalSteps, 0.25) OVER(),
       PERCENTILE_CONT(VeryActiveMinutes, 0.25) OVER(),
       PERCENTILE_CONT(FairlyActiveMinutes, 0.25) OVER(),
       PERCENTILE_CONT(LightlyActiveMinutes, 0.25) OVER(),
       PERCENTILE_CONT(SedentaryMinutes, 0.25) OVER()
  FROM `gac-bellabeat.activity.daily`
  LIMIT 1) 
  UNION ALL 
(SELECT 66,
       '50%',
       PERCENTILE_CONT(TotalSteps, 0.50) OVER(),
       PERCENTILE_CONT(VeryActiveMinutes, 0.50) OVER(),
       PERCENTILE_CONT(FairlyActiveMinutes, 0.50) OVER(),
       PERCENTILE_CONT(LightlyActiveMinutes, 0.50) OVER(),
       PERCENTILE_CONT(SedentaryMinutes, 0.50) OVER()
  FROM `gac-bellabeat.activity.daily`
  LIMIT 1) 
  UNION ALL 
(SELECT 7,
       '75%',
       PERCENTILE_CONT(TotalSteps, 0.75) OVER(),
       PERCENTILE_CONT(VeryActiveMinutes, 0.75) OVER(),
       PERCENTILE_CONT(FairlyActiveMinutes, 0.75) OVER(),
       PERCENTILE_CONT(LightlyActiveMinutes, 0.75) OVER(),
       PERCENTILE_CONT(SedentaryMinutes, 0.75) OVER()
  FROM `gac-bellabeat.activity.daily`
 LIMIT 1) 
 UNION ALL
SELECT 8,
       'Max',
       MAX(TotalSteps),
       MAX(VeryActiveMinutes),
       MAX(FairlyActiveMinutes),
       MAX(LightlyActiveMinutes),
       MAX(SedentaryMinutes)
  FROM `gac-bellabeat.activity.daily`
ORDER BY Index ASC;

Downloading: 100%|██████████| 8/8 [00:01<00:00,  5.67rows/s]


,Index,Statistic,total_steps,very_active_minutes,fairly_active_minutes,lightly_active_minutes,sedentary_minutes
0,1,Count,940.00,940.0,940.0,940.0,940.00
1,2,Mean,7637.90,21.2,13.6,192.8,991.20
2,3,STD,5084.40,32.8,20.0,109.1,301.10
3,4,Min,0.00,0.0,0.0,0.0,0.00
4,5,25%,3789.75,0.0,0.0,127.0,729.75
5,7,75%,10727.00,32.0,19.0,264.0,1229.50
6,8,Max,36019.00,210.0,143.0,518.0,1440.00
7,66,50%,7405.50,4.0,6.0,199.0,1057.50


##### Create new DayOfWeek column extracted from ActivityDate to faciliate analysis

In [7]:
%%bigquery
SELECT Id,
       ActivityDate,
       FORMAT_DATE('%a', ActivityDate) AS DayOfWeek,
       TotalSteps,
       VeryActiveMinutes,
       FairlyActiveMinutes,
       LightlyActiveMinutes,
       SedentaryMinutes, 
       Calories
FROM `gac-bellabeat.activity.daily`;

Downloading: 100%|██████████| 940/940 [00:01<00:00, 659.67rows/s]


,Id,ActivityDate,DayOfWeek,TotalSteps,VeryActiveMinutes,FairlyActiveMinutes,LightlyActiveMinutes,SedentaryMinutes,Calories
0,1624580081,2016-05-01,Sun,36019,186,63,171,1020,2690
1,1644430081,2016-04-14,Thu,11037,5,58,252,1125,3226
2,1644430081,2016-04-19,Tue,11256,5,58,278,1099,3300
3,1644430081,2016-04-28,Thu,9405,3,53,227,1157,3108
4,1644430081,2016-04-30,Sat,18213,9,71,402,816,3846
...,...,...,...,...,...,...,...,...,...
935,1844505072,2016-04-20,Wed,8,0,0,1,1439,1349
936,4020332650,2016-04-17,Sun,16,0,0,2,1438,1990
937,4319703577,2016-05-12,Thu,17,0,0,2,0,257
938,6775888955,2016-05-03,Tue,9,0,0,1,1439,1843


### Hourly Activity Data

There are three tables (hourly_steps, hourly_calories, hourly_intensities) recording hourly data. Let's check if there are any differences among the three tables prior to merging them on the columns Id and ActivityHour as a compound key. We'll do that by comparing a pair of tables at a time and using EXCEPT, reverse the positions of the tables, and then UNION ALL the result. If there are any combinations of Id, Activity that are only present in one of the tables, we should see that combination in our query results.

In [8]:
%%bigquery
(SELECT hs.Id, hs.ActivityHour
   FROM `gac-bellabeat.activity.hourly_steps` hs
 EXCEPT DISTINCT
 SELECT hc.Id, hc.ActivityHour
  FROM `gac-bellabeat.activity.hourly_calories` hc)
UNION ALL
(SELECT hc.Id, hc.ActivityHour
   FROM `gac-bellabeat.activity.hourly_calories` hc
 EXCEPT DISTINCT
 SELECT hs.Id, hs.ActivityHour
  FROM `gac-bellabeat.activity.hourly_steps` hs);

Query complete after 0.01s: 100%|██████████| 7/7 [00:00<00:00, 1149.21query/s]                        
Downloading: 0rows [00:00, ?rows/s]


,Id,ActivityHour


In [9]:
%%bigquery
(SELECT hi.Id, hi.ActivityHour
   FROM `gac-bellabeat.activity.hourly_intensities` hi
 EXCEPT DISTINCT
 SELECT hc.Id, hc.ActivityHour
   FROM `gac-bellabeat.activity.hourly_calories` hc)
UNION ALL
(SELECT hc.Id, hc.ActivityHour
   FROM `gac-bellabeat.activity.hourly_calories` hc
 EXCEPT DISTINCT
 SELECT hi.Id, hi.ActivityHour
   FROM `gac-bellabeat.activity.hourly_intensities` hi);

Query complete after 0.01s: 100%|██████████| 7/7 [00:00<00:00, 1436.48query/s]                        
Downloading: 0rows [00:00, ?rows/s]


,Id,ActivityHour


Since there are no resulting rows from our queries, we can safely merge all three tables on the compound keys Id, ActivityHour.

In [10]:
%%bigquery
SELECT *
  FROM `gac-bellabeat.activity.hourly_intensities` hi
  JOIN `gac-bellabeat.activity.hourly_steps` hs
    ON hi.Id = hs.Id
   AND hi.ActivityHour = hs.ActivityHour
  JOIN  `gac-bellabeat.activity.hourly_calories` hc
    ON hi.Id = hc.Id
   AND hi.ActivityHour = hc.ActivityHour  
 ORDER BY hi.Id, hi.ActivityHour

Downloading: 100%|██████████| 22099/22099 [00:01<00:00, 13141.81rows/s]


,Id,ActivityHour,TotalIntensity,AverageIntensity,Id_1,ActivityHour_1,StepTotal,Id_2,ActivityHour_2,Calories
0,1503960366,2016-04-12 00:00:00+00:00,20,0.333333,1503960366,2016-04-12 00:00:00+00:00,373,1503960366,2016-04-12 00:00:00+00:00,81
1,1503960366,2016-04-12 01:00:00+00:00,8,0.133333,1503960366,2016-04-12 01:00:00+00:00,160,1503960366,2016-04-12 01:00:00+00:00,61
2,1503960366,2016-04-12 02:00:00+00:00,7,0.116667,1503960366,2016-04-12 02:00:00+00:00,151,1503960366,2016-04-12 02:00:00+00:00,59
3,1503960366,2016-04-12 03:00:00+00:00,0,0.000000,1503960366,2016-04-12 03:00:00+00:00,0,1503960366,2016-04-12 03:00:00+00:00,47
4,1503960366,2016-04-12 04:00:00+00:00,0,0.000000,1503960366,2016-04-12 04:00:00+00:00,0,1503960366,2016-04-12 04:00:00+00:00,48
...,...,...,...,...,...,...,...,...,...,...
22094,8877689391,2016-05-12 10:00:00+00:00,12,0.200000,8877689391,2016-05-12 10:00:00+00:00,514,8877689391,2016-05-12 10:00:00+00:00,126
22095,8877689391,2016-05-12 11:00:00+00:00,29,0.483333,8877689391,2016-05-12 11:00:00+00:00,1407,8877689391,2016-05-12 11:00:00+00:00,192
22096,8877689391,2016-05-12 12:00:00+00:00,93,1.550000,8877689391,2016-05-12 12:00:00+00:00,3135,8877689391,2016-05-12 12:00:00+00:00,321
22097,8877689391,2016-05-12 13:00:00+00:00,6,0.100000,8877689391,2016-05-12 13:00:00+00:00,307,8877689391,2016-05-12 13:00:00+00:00,101


To make take this merged table easier to work with for analysis, we'll remove the duplicate columns (Id, ActivityHour) and extract separate HourOfDay and DayOfWeek columns from ActivityHour, and then save it as a view (`gac-bellabeat.activity.hourly`) to access later in our analysis.

In [11]:
%%bigquery
SELECT hi.Id,
       hi.ActivityHour,
       EXTRACT(hour FROM hi.ActivityHour) AS HourOfDay,
       FORMAT_DATE('%a', hi.ActivityHour) AS DayOfWeek,
       TotalIntensity,
       AverageIntensity,
       StepTotal,
       Calories
  FROM `gac-bellabeat.activity.hourly_intensities` hi
  JOIN `gac-bellabeat.activity.hourly_steps` hs
    ON hi.Id = hs.Id
   AND hi.ActivityHour = hs.ActivityHour
  JOIN  `gac-bellabeat.activity.hourly_calories` hc
    ON hi.Id = hc.Id
   AND hi.ActivityHour = hc.ActivityHour  
 ORDER BY hi.Id, hi.ActivityHour
              

Downloading: 100%|██████████| 22099/22099 [00:01<00:00, 13129.93rows/s]


,Id,ActivityHour,HourOfDay,DayOfWeek,TotalIntensity,AverageIntensity,StepTotal,Calories
0,1503960366,2016-04-12 00:00:00+00:00,0,Tue,20,0.333333,373,81
1,1503960366,2016-04-12 01:00:00+00:00,1,Tue,8,0.133333,160,61
2,1503960366,2016-04-12 02:00:00+00:00,2,Tue,7,0.116667,151,59
3,1503960366,2016-04-12 03:00:00+00:00,3,Tue,0,0.000000,0,47
4,1503960366,2016-04-12 04:00:00+00:00,4,Tue,0,0.000000,0,48
...,...,...,...,...,...,...,...,...
22094,8877689391,2016-05-12 10:00:00+00:00,10,Thu,12,0.200000,514,126
22095,8877689391,2016-05-12 11:00:00+00:00,11,Thu,29,0.483333,1407,192
22096,8877689391,2016-05-12 12:00:00+00:00,12,Thu,93,1.550000,3135,321
22097,8877689391,2016-05-12 13:00:00+00:00,13,Thu,6,0.100000,307,101


Next, we'll save this table as a view to use later in our analysis.

Let's run our summary stat panel on this data. 

In [12]:
%%bigquery
SELECT 1 AS Index, 
      'Count' AS Statistic,
       COUNT(TotalIntensity) AS total_intensity,
       COUNT(AverageIntensity) AS avg_intensity,
       COUNT(StepTotal) AS step_total,
       COUNT(Calories) AS calories
  FROM `gac-bellabeat.activity.hourly`
 UNION ALL
SELECT 2, 
      'Mean',
       ROUND(AVG(TotalIntensity), 1),
       ROUND(AVG(AverageIntensity), 1),
       ROUND(AVG(StepTotal), 1),
       ROUND(AVG(Calories), 1)
  FROM `gac-bellabeat.activity.hourly`
  UNION ALL
(SELECT 3,
       'STD',
       ROUND(STDDEV_POP(TotalIntensity) OVER(), 1),
       ROUND(STDDEV_POP(AverageIntensity) OVER(), 1),
       ROUND(STDDEV_POP(StepTotal) OVER(), 1),
       ROUND(STDDEV_POP(Calories) OVER(), 1)
  FROM `gac-bellabeat.activity.hourly`
 LIMIT 1)    
  UNION ALL
SELECT 4,
       'Min',
       MIN(TotalIntensity) AS min_total_steps,
       MIN(AverageIntensity) AS min_avg_intensity,
       MIN(StepTotal) AS min_step_total,
       MIN(Calories) AS min_calories
  FROM `gac-bellabeat.activity.hourly`
  UNION ALL
(SELECT 5,
       '25%',
       PERCENTILE_CONT(TotalIntensity, 0.25) OVER(),
       PERCENTILE_CONT(AverageIntensity, 0.25) OVER(),
       PERCENTILE_CONT(StepTotal, 0.25) OVER(),
       PERCENTILE_CONT(Calories, 0.25) OVER()
  FROM `gac-bellabeat.activity.hourly`
  LIMIT 1) 
  UNION ALL 
(SELECT 6,
       '50%',
       PERCENTILE_CONT(TotalIntensity, 0.50) OVER(),
       PERCENTILE_CONT(AverageIntensity, 0.50) OVER(),
       PERCENTILE_CONT(StepTotal, 0.50) OVER(),
       PERCENTILE_CONT(Calories, 0.50) OVER()
  FROM `gac-bellabeat.activity.hourly`
  LIMIT 1) 
  UNION ALL 
(SELECT 7,
       '75%',
       PERCENTILE_CONT(TotalIntensity, 0.75) OVER(),
       PERCENTILE_CONT(AverageIntensity, 0.75) OVER(),
       PERCENTILE_CONT(StepTotal, 0.75) OVER(),
       PERCENTILE_CONT(Calories, 0.75) OVER()
  FROM `gac-bellabeat.activity.hourly`
  LIMIT 1) 
UNION ALL
SELECT 8,
       'Max',
       MAX(TotalIntensity) AS max_total_intensity,
       MAX(AverageIntensity) AS max_avg_intensity,
       MAX(StepTotal) AS max_step_total,
       MAX(Calories) AS max_calories
  FROM `gac-bellabeat.activity.hourly`
ORDER BY Index ASC;

Downloading: 100%|██████████| 8/8 [00:01<00:00,  4.92rows/s]


,Index,Statistic,total_intensity,avg_intensity,step_total,calories
0,1,Count,22099.0,22099.000000,22099.0,22099.0
1,2,Mean,12.0,0.200000,320.2,97.4
2,3,STD,21.1,0.400000,690.4,60.7
3,4,Min,0.0,0.000000,0.0,42.0
4,5,25%,0.0,0.000000,0.0,63.0
5,6,50%,3.0,0.050000,40.0,83.0
6,7,75%,16.0,0.266667,357.0,108.0
7,8,Max,180.0,3.000000,10554.0,948.0


Let's also validate the number of unique individuals.

In [13]:
%%bigquery
SELECT COUNT(DISTINCT Id) AS user_count
 FROM `gac-bellabeat.activity.hourly`;

Downloading: 100%|██████████| 1/1 [00:01<00:00,  1.47s/rows]


,user_count
0,33


### Sleep Data

Let's inspect the sleep monitoring data and count the number of unique users that contributed sleep tracking data.

In [14]:
%%bigquery
SELECT COUNT(DISTINCT Id) AS user_count
  FROM `gac-bellabeat.activity.sleep`;
    

Downloading: 100%|██████████| 1/1 [00:01<00:00,  1.42s/rows]


,user_count
0,24


In [15]:
%%bigquery
SELECT *
FROM `gac-bellabeat.activity.sleep`
LIMIT 1;

Downloading: 100%|██████████| 1/1 [00:01<00:00,  1.42s/rows]


,Id,SleepDay,TotalSleepRecords,TotalMinutesAsleep,TotalTimeInBed
0,1503960366,2016-04-12 00:00:00+00:00,1,327,346


We see that only 24 out of 33 users tracked their sleep data. Let's also run a stat summary panel. 

In [16]:
%%bigquery
SELECT 1 AS Index, 
      'Count' AS Statistic,
       COUNT(TotalSleepRecords) AS total_sleep_records,
       COUNT(TotalMinutesAsleep) AS total_minutes_asleep,
       COUNT(TotalTimeinBed) AS total_time_in_bed
  FROM `gac-bellabeat.activity.sleep`
 UNION ALL
SELECT 2, 
      'Mean',
       ROUND(AVG(TotalSleepRecords), 1),
       ROUND(AVG(TotalMinutesAsleep), 1),
       ROUND(AVG(TotalTimeinBed), 1)
  FROM `gac-bellabeat.activity.sleep`
 UNION ALL
(SELECT 3,
       'STD',
       ROUND(STDDEV_POP(TotalSleepRecords) OVER(), 1),
       ROUND(STDDEV_POP(TotalMinutesAsleep) OVER(), 1),
       ROUND(STDDEV_POP(TotalTimeinBed) OVER(), 1)
  FROM `gac-bellabeat.activity.sleep`
 LIMIT 1)
 UNION ALL
(SELECT 4,
       'Min',
       MIN(TotalSleepRecords),
       MIN(TotalMinutesAsleep),
       MIN(TotalTimeinBed)
  FROM `gac-bellabeat.activity.sleep`)
  UNION ALL
(SELECT 5,
       '25%',
       PERCENTILE_CONT(TotalSleepRecords, 0.25) OVER(),
       PERCENTILE_CONT(TotalMinutesAsleep, 0.25) OVER(),
       PERCENTILE_CONT(TotalTimeinBed, 0.25) OVER()
  FROM `gac-bellabeat.activity.sleep`
  LIMIT 1) 
  UNION ALL 
(SELECT 6,
       '50%',
       PERCENTILE_CONT(TotalSleepRecords, 0.50) OVER(),
       PERCENTILE_CONT(TotalMinutesAsleep, 0.50) OVER(),
       PERCENTILE_CONT(TotalTimeinBed, 0.50) OVER()
  FROM `gac-bellabeat.activity.sleep`
  LIMIT 1) 
  UNION ALL 
(SELECT 7,
       '75%',
       PERCENTILE_CONT(TotalSleepRecords, 0.75) OVER(),
       PERCENTILE_CONT(TotalMinutesAsleep, 0.75) OVER(),
       PERCENTILE_CONT(TotalTimeinBed, 0.75) OVER()
  FROM `gac-bellabeat.activity.sleep`
  LIMIT 1) 
  UNION ALL
(SELECT 8,
       'Max',
       MAX(TotalSleepRecords),
       MAX(TotalMinutesAsleep),
       MAX(TotalTimeinBed)
  FROM `gac-bellabeat.activity.sleep`)
ORDER BY Index ASC;


Downloading: 100%|██████████| 8/8 [00:01<00:00,  5.60rows/s]


,Index,Statistic,total_sleep_records,total_minutes_asleep,total_time_in_bed
0,1,Count,413.0,413.0,413.0
1,2,Mean,1.1,419.5,458.6
2,3,STD,0.3,118.2,126.9
3,4,Min,1.0,58.0,61.0
4,5,25%,1.0,361.0,403.0
5,6,50%,1.0,433.0,463.0
6,7,75%,1.0,490.0,526.0
7,8,Max,3.0,796.0,961.0


Let's also create a DayOfWeek column extracted from SleepDay to help with analysis later.

In [17]:
%%bigquery
SELECT SleepDay,
       FORMAT_DATE('%a', SleepDay) AS DayOfWeek,
       TotalSleepRecords,
       TotalMinutesAsleep,
       TotalTimeInBed
 FROM `gac-bellabeat.activity.sleep`;

Downloading: 100%|██████████| 413/413 [00:01<00:00, 292.15rows/s]


,SleepDay,DayOfWeek,TotalSleepRecords,TotalMinutesAsleep,TotalTimeInBed
0,2016-04-12 00:00:00+00:00,Tue,1,327,346
1,2016-04-15 00:00:00+00:00,Fri,1,412,442
2,2016-04-17 00:00:00+00:00,Sun,1,700,712
3,2016-04-19 00:00:00+00:00,Tue,1,304,320
4,2016-04-20 00:00:00+00:00,Wed,1,360,377
...,...,...,...,...,...
408,2016-04-17 00:00:00+00:00,Sun,2,525,591
409,2016-05-07 00:00:00+00:00,Sat,2,459,513
410,2016-04-12 00:00:00+00:00,Tue,3,750,775
411,2016-04-24 00:00:00+00:00,Sun,3,552,595


### Weight Data

We'll now inspect the weight table and count how many unique users contributed data.

In [18]:
%%bigquery
SELECT COUNT(DISTINCT Id) AS user_count 
  FROM `gac-bellabeat.activity.weight`;

Downloading: 100%|██████████| 1/1 [00:01<00:00,  1.42s/rows]


,user_count
0,8


Let's run our stat summary panel.

In [19]:
%%bigquery
SELECT 1 AS Index, 
      'Count' AS Statistic,
       COUNT(WeightKg) AS weight_kg,
       COUNT(WeightPounds) AS weight_pounds,
       COUNT(Fat) AS fat,
       COUNT(BMI) AS bmi  
  FROM `gac-bellabeat.activity.weight`
 UNION ALL
SELECT 2, 
      'Mean',
       ROUND(AVG(WeightKg), 1),
       ROUND(AVG(WeightPounds), 1),
       ROUND(AVG(Fat), 1),
       ROUND(AVG(BMI), 1),
  FROM `gac-bellabeat.activity.weight`
 UNION ALL
(SELECT 3,
       'STD',
       ROUND(STDDEV_POP(WeightKg) OVER(), 1),
       ROUND(STDDEV_POP(WeightPounds) OVER(), 1),
       ROUND(STDDEV_POP(Fat) OVER(), 1),
       ROUND(STDDEV_POP(BMI) OVER(), 1)
  FROM `gac-bellabeat.activity.weight`
 LIMIT 1)
 UNION ALL
(SELECT 4,
       'Min',
       MIN(WeightKg),
       MIN(WeightPounds),
       MIN(Fat),
       MIN(BMI)
  FROM `gac-bellabeat.activity.weight`)
  UNION ALL
(SELECT 5,
       '25%',
       PERCENTILE_CONT(WeightKg, 0.25) OVER(),
       PERCENTILE_CONT(WeightPounds, 0.25) OVER(),
       PERCENTILE_CONT(Fat, 0.25) OVER(),
       PERCENTILE_CONT(BMI, 0.25) OVER()
  FROM `gac-bellabeat.activity.weight`
  LIMIT 1) 
  UNION ALL 
(SELECT 6,
       '50%',
       PERCENTILE_CONT(WeightKg, 0.50) OVER(),
       PERCENTILE_CONT(WeightPounds, 0.50) OVER(),
       PERCENTILE_CONT(Fat, 0.50) OVER(),
       PERCENTILE_CONT(BMI, 0.50) OVER()
  FROM `gac-bellabeat.activity.weight`
  LIMIT 1) 
  UNION ALL 
(SELECT 7,
       '75%',
       PERCENTILE_CONT(WeightKg, 0.75) OVER(),
       PERCENTILE_CONT(WeightPounds, 0.75) OVER(),
       PERCENTILE_CONT(Fat, 0.75) OVER(),
       PERCENTILE_CONT(BMI, 0.75) OVER()
  FROM `gac-bellabeat.activity.weight`
  LIMIT 1) 
  UNION ALL
(SELECT 8,
       'Max',
       MAX(WeightKg),
       MAX(WeightPounds),
       MAX(Fat),
       MAX(BMI)
  FROM `gac-bellabeat.activity.weight`)
ORDER BY Index ASC;

Downloading: 100%|██████████| 8/8 [00:01<00:00,  5.40rows/s]


,Index,Statistic,weight_kg,weight_pounds,fat,bmi
0,1,Count,67.000000,67.000000,2.00,67.000000
1,2,Mean,72.000000,158.800000,23.50,25.200000
2,3,STD,13.800000,30.500000,1.50,3.000000
3,4,Min,52.599998,115.963147,22.00,21.450001
4,5,25%,61.400002,135.363832,22.75,23.959999
5,6,50%,62.500000,137.788914,23.50,24.389999
6,7,75%,85.049999,187.503152,24.25,25.559999
7,8,Max,133.500000,294.317120,25.00,47.540001


Add info here about how healthy this cohort seems to be, but sample is too small. 

## ANALYZE 

In this section, we'll get a clearer sense of the users' behavior by analyzing their physical activity, sleep, and weight data.

We'll break this section down into 3 parts:
- User Engagement
- User Habits
- User Profile



## **User Engagement**

We'll first look at how Fitbit users made use of the activity, sleep, and weight tracking capabilities on the smart device.  

### **Engagement** | Activity Data

To measure level of engagement, we'll look at how often users wore their trackers during the 1-month period. 

We'll group users into 3 categories:
- **High**: usage greater than 3 out of 4 weeks (>20 days)
- **Moderate**: usage at least 2 weeks but less than 3 weeks (14-20 days)
- **Low**: usage less than 2 weeks (<14 days)

In [68]:
%%bigquery
SELECT Id,
       MIN(ActivityDate) AS first_record,
       MAX(ActivityDate) AS last_record,
       DATE_DIFF(MAX(ActivityDate), MIN(ActivityDate), DAY) + 1 AS ActiveWindowDays, -- add 1 to include usage on final day
       COUNT(ActivityDate) AS record_count,
       CASE 
        WHEN COUNT(ActivityDate) >= 21 THEN 'High'
        WHEN COUNT(ActivityDate) >= 14 THEN 'Moderate'
        ELSE 'Low'
       END AS UsageLevel
FROM `gac-bellabeat.activity.daily`
GROUP BY 1
ORDER BY 4 DESC, 5 DESC, 2, 3;

Downloading: 100%|██████████| 33/33 [00:01<00:00, 22.13rows/s]


,Id,first_record,last_record,ActiveWindowDays,record_count,UsageLevel
0,1624580081,2016-04-12,2016-05-12,31,31,High
1,2022484408,2016-04-12,2016-05-12,31,31,High
2,4319703577,2016-04-12,2016-05-12,31,31,High
3,4388161847,2016-04-12,2016-05-12,31,31,High
4,4702921684,2016-04-12,2016-05-12,31,31,High
5,6962181067,2016-04-12,2016-05-12,31,31,High
6,7086361926,2016-04-12,2016-05-12,31,31,High
7,8583815059,2016-04-12,2016-05-12,31,31,High
8,1844505072,2016-04-12,2016-05-12,31,31,High
9,1927972279,2016-04-12,2016-05-12,31,31,High


In [77]:
%%bigquery
WITH usage_categories AS (
    SELECT Id,
           MIN(ActivityDate) AS first_record,
           MAX(ActivityDate) AS last_record,
           DATE_DIFF(MAX(ActivityDate), MIN(ActivityDate), DAY) + 1 AS ActiveWindowDays, -- add 1 to include usage on final day
           COUNT(ActivityDate) AS record_count,
           CASE 
            WHEN COUNT(ActivityDate) >= 21 THEN 'High'
            WHEN COUNT(ActivityDate) >= 14 THEN 'Moderate'
            ELSE 'Low'
           END AS UsageLevel
     FROM `gac-bellabeat.activity.daily`
    GROUP BY 1
)
SELECT SUM(CASE WHEN uc.UsageLevel = 'High' THEN 1 ELSE 0 END) AS high_usage,
       SUM(CASE WHEN uc.UsageLevel = 'Moderate' THEN 1 ELSE 0 END) AS moderate_usage,
       SUM(CASE WHEN uc.UsageLevel = 'Low' THEN 1 ELSE 0 END) AS low_usage, 
       COUNT(*) AS total_users
FROM usage_categories uc;

Downloading: 100%|██████████| 1/1 [00:01<00:00,  1.56s/rows]


,high_usage,moderate_usage,low_usage,total_users
0,29,3,1,33


Within this group of users, there was a high level of usage with 29 out of 33 users *wearing* the Fitbit and also *tracking activity* for at least 21 days out of 31. 

In other words, out of 33 users, only 4 wore the tracker less than 3 weeks during the 1-month period indicating that most users consistently wore the device and used it to track their physical activity. 

Next, we'll look at sleep tracking. 

### **Engagement** | Sleep Tracking

In [117]:
%%bigquery
WITH sleep_record_count AS (
    SELECT Id,
           SUM(TotalSleepRecords) AS record_count
     FROM `gac-bellabeat.activity.sleep`
    GROUP BY 1
),
active_window AS (
    SELECT Id,
           MIN(SleepDay) AS first_record,
           MAX(SleepDay) AS last_record,
           DATE_DIFF(MAX(SleepDay), MIN(SleepDay), DAY) + 1 AS ActiveWindowDays, -- add 1 to include usage on final day
     FROM `gac-bellabeat.activity.sleep`
    GROUP BY 1
)
SELECT aw.Id,
       aw.first_record,
       aw.last_record,
       aw.ActiveWindowDays,
       src.record_count,
       CASE 
        WHEN src.record_count >= 21 THEN 'High' 
        WHEN src.record_count >= 14 THEN 'Moderate'
        ELSE 'Low'
       END AS UsageLevel
 FROM active_window aw
 JOIN sleep_record_count src
   ON aw.Id = src.Id
ORDER BY 4 DESC, 5 DESC, 6;

Downloading: 100%|██████████| 24/24 [00:01<00:00, 20.09rows/s]


,Id,first_record,last_record,ActiveWindowDays,record_count,UsageLevel
0,4445114986,2016-04-12 00:00:00+00:00,2016-05-12 00:00:00+00:00,31,39,High
1,5553957443,2016-04-12 00:00:00+00:00,2016-05-12 00:00:00+00:00,31,38,High
2,8378563200,2016-04-12 00:00:00+00:00,2016-05-12 00:00:00+00:00,31,36,High
3,6962181067,2016-04-12 00:00:00+00:00,2016-05-12 00:00:00+00:00,31,34,High
4,4702921684,2016-04-12 00:00:00+00:00,2016-05-12 00:00:00+00:00,31,30,High
5,2026352035,2016-04-12 00:00:00+00:00,2016-05-12 00:00:00+00:00,31,28,High
6,7086361926,2016-04-12 00:00:00+00:00,2016-05-12 00:00:00+00:00,31,24,High
7,1503960366,2016-04-12 00:00:00+00:00,2016-05-11 00:00:00+00:00,30,27,High
8,5577150313,2016-04-12 00:00:00+00:00,2016-05-11 00:00:00+00:00,30,27,High
9,3977333714,2016-04-12 00:00:00+00:00,2016-05-10 00:00:00+00:00,29,32,High


In [78]:
%%bigquery
WITH usage_categories AS (
    SELECT Id,
           MIN(SleepDay) AS first_record,
           MAX(SleepDay) AS last_record,
           DATE_DIFF(MAX(SleepDay), MIN(SleepDay), DAY) + 1 AS ActiveWindowDays, -- add 1 to include usage on final day
           COUNT(SleepDay) AS record_count,
           CASE 
            WHEN COUNT(SleepDay) >= 21 THEN 'High' 
            WHEN COUNT(SleepDay) >= 14 THEN 'Moderate'
            ELSE 'Low'
           END AS UsageLevel
     FROM `gac-bellabeat.activity.sleep`
    GROUP BY 1
)
SELECT SUM(CASE WHEN uc.UsageLevel = 'High' THEN 1 ELSE 0 END) AS high_usage,
       SUM(CASE WHEN uc.UsageLevel = 'Moderate' THEN 1 ELSE 0 END) AS moderate_usage,
       SUM(CASE WHEN uc.UsageLevel = 'Low' THEN 1 ELSE 0 END) AS low_usage, 
       COUNT(*) AS total_users
FROM usage_categories uc;

Downloading: 100%|██████████| 1/1 [00:01<00:00,  1.73s/rows]


,high_usage,moderate_usage,low_usage,total_users
0,12,3,9,24


In [120]:
%%bigquery
WITH sleep_record_count AS (
    SELECT Id,
           SUM(TotalSleepRecords) AS record_count
     FROM `gac-bellabeat.activity.sleep`
    GROUP BY 1
),
active_window AS (
    SELECT Id,
           MIN(SleepDay) AS first_record,
           MAX(SleepDay) AS last_record,
           DATE_DIFF(MAX(SleepDay), MIN(SleepDay), DAY) + 1 AS ActiveWindowDays, -- add 1 to include usage on final day
     FROM `gac-bellabeat.activity.sleep`
    GROUP BY 1
),
usage_categories AS (
    SELECT aw.Id,
           aw.first_record,
           aw.last_record,
           aw.ActiveWindowDays,
           src.record_count,
           CASE 
            WHEN src.record_count >= 21 THEN 'High' 
            WHEN src.record_count >= 14 THEN 'Moderate'
            ELSE 'Low'
           END AS UsageLevel
     FROM active_window aw
     JOIN sleep_record_count src
       ON aw.Id = src.Id
)
SELECT SUM(CASE WHEN uc.UsageLevel = 'High' THEN 1 ELSE 0 END) AS high_usage,
       SUM(CASE WHEN uc.UsageLevel = 'Moderate' THEN 1 ELSE 0 END) AS moderate_usage,
       SUM(CASE WHEN uc.UsageLevel = 'Low' THEN 1 ELSE 0 END) AS low_usage, 
       COUNT(*) AS total_users
 FROM usage_categories uc;

Downloading: 100%|██████████| 1/1 [00:01<00:00,  1.49s/rows]


,high_usage,moderate_usage,low_usage,total_users
0,13,2,9,24


With sleep tracking we have a slightly different picture. Only **80% of of the group (24 out of 33 users)** took advantage of this functionality. This potentially points to users in this group not having an interest in monitoring their sleep,  not aware of this functionality, or unable to activate this capability.

Additionally, within this subset, a smaller proportion (13 out of 24 users) used the device to monitor sleep on a consistent basis when compared to tracking physical activity. Even when the active window of using the device (defined as the period from the first to last record) is relatively long, there are signs of low usage given by the low `record_count`. Note: `record_count` includes naps taken that are greater than 60 minutes. This explains why in some cases the `record_count` exceeds the number of days during the active window.  

This lower usage may point to issues related to usefulness or understanding of the data by the user, or even possibly difficulty or discomfort in wearing the device.   

In the following section, we'll look at user engagement with weight tracking.


### **Engagement** | Weight Tracking

In [121]:
%%bigquery
SELECT *
FROM `gac-bellabeat.activity.weight`

Downloading: 100%|██████████| 67/67 [00:01<00:00, 46.85rows/s]


,Id,Date,WeightKg,WeightPounds,Fat,BMI,IsManualReport,LogId
0,4558609924,2016-04-18 23:59:59+00:00,69.699997,153.662190,<NA>,27.250000,True,1461023999000
1,4558609924,2016-04-25 23:59:59+00:00,70.300003,154.984977,<NA>,27.459999,True,1461628799000
2,4558609924,2016-05-01 23:59:59+00:00,69.900002,154.103125,<NA>,27.320000,True,1462147199000
3,4558609924,2016-05-02 23:59:59+00:00,69.199997,152.559879,<NA>,27.040001,True,1462233599000
4,4558609924,2016-05-09 23:59:59+00:00,69.099998,152.339420,<NA>,27.000000,True,1462838399000
...,...,...,...,...,...,...,...,...
62,6962181067,2016-04-18 23:59:59+00:00,61.200001,134.922906,<NA>,23.889999,True,1461023999000
63,6962181067,2016-04-27 23:59:59+00:00,61.200001,134.922906,<NA>,23.889999,True,1461801599000
64,6962181067,2016-04-28 23:59:59+00:00,61.200001,134.922906,<NA>,23.889999,True,1461887999000
65,6962181067,2016-05-07 23:59:59+00:00,61.200001,134.922906,<NA>,23.889999,True,1462665599000


In [122]:
%%bigquery
SELECT Id,
       MIN(Date) AS first_record,
       MAX(Date) AS last_record,
       DATE_DIFF(MAX(Date), MIN(Date), DAY) + 1 AS ActiveWindowDays, -- add 1 to include usage on final day
       COUNT(LogId) AS record_count,
       SUM(CASE WHEN IsManualReport IS True THEN 1 ELSE 0 END) AS manual_report_count,
       SUM(CASE WHEN IsManualReport IS False THEN 1 ELSE 0 END) AS auto_report_count,
       CASE 
        WHEN COUNT(LogId) >= 21 THEN 'High'
        WHEN COUNT(LogId) >= 14 THEN 'Moderate'
        ELSE 'Low'
       END AS UsageLevel,
 FROM `gac-bellabeat.activity.weight`
GROUP BY Id
ORDER BY 4 DESC, 5 DESC, 2, 3;

Downloading: 100%|██████████| 8/8 [00:01<00:00,  5.62rows/s]


,Id,first_record,last_record,ActiveWindowDays,record_count,manual_report_count,auto_report_count,UsageLevel
0,6962181067,2016-04-12 23:59:59+00:00,2016-05-12 23:59:59+00:00,31,30,30,0,High
1,8877689391,2016-04-12 06:47:11+00:00,2016-05-12 06:42:53+00:00,30,24,0,24,High
2,4558609924,2016-04-18 23:59:59+00:00,2016-05-09 23:59:59+00:00,22,5,5,0,Low
3,2873212765,2016-04-21 23:59:59+00:00,2016-05-12 23:59:59+00:00,22,2,2,0,Low
4,4319703577,2016-04-17 23:59:59+00:00,2016-05-04 23:59:59+00:00,18,2,2,0,Low
5,1503960366,2016-05-02 23:59:59+00:00,2016-05-03 23:59:59+00:00,2,2,2,0,Low
6,1927972279,2016-04-13 01:08:52+00:00,2016-04-13 01:08:52+00:00,1,1,0,1,Low
7,5577150313,2016-04-17 09:17:55+00:00,2016-04-17 09:17:55+00:00,1,1,0,1,Low


In [123]:
%%bigquery
WITH usage_categories AS ( 
    SELECT Id,
           MIN(Date) AS first_record,
           MAX(Date) AS last_record,
           DATE_DIFF(MAX(Date), MIN(Date), DAY) + 1 AS ActiveUsageDays, -- add 1 to include usage on final day
           COUNT(LogId) AS record_count,
           SUM(CASE WHEN IsManualReport IS True THEN 1 ELSE 0 END) AS manual_report_count,
           SUM(CASE WHEN IsManualReport IS False THEN 1 ELSE 0 END) AS auto_report_count,
           CASE 
            WHEN COUNT(LogId) >= 21 THEN 'High'
            WHEN COUNT(LogId) >= 14 THEN 'Moderate'
            ELSE 'Low'
           END AS UsageLevel
    FROM `gac-bellabeat.activity.weight`
    GROUP BY Id
) 
SELECT SUM(CASE WHEN uc.UsageLevel = 'High' THEN 1 ELSE 0 END) AS high_usage,
       SUM(CASE WHEN uc.UsageLevel = 'Moderate' THEN 1 ELSE 0 END) AS moderate_usage,
       SUM(CASE WHEN uc.UsageLevel = 'Low' THEN 1 ELSE 0 END) AS low_usage, 
       SUM(CASE WHEN manual_report_count > 0 THEN 1 ELSE 0 END) AS manual_user_count,
       SUM(CASE WHEN auto_report_count >0 THEN 1 ELSE 0 END) AS auto_user_count,
       COUNT(*) AS total_users
FROM usage_categories uc;

Downloading: 100%|██████████| 1/1 [00:01<00:00,  1.42s/rows]


,high_usage,moderate_usage,low_usage,manual_user_count,auto_user_count,total_users
0,2,0,6,5,3,8


Compared to activity and sleep tracking, only a few (8 out of 33 users) made use of the weight tracking functionality. This suggests that users may not be interested in monitoring weight, unaware of this capability, or possibly find that the available input methods are troublesome. 

Weight logs must be input manually by the user or separately synced with a digital scale (e.g. Withings, Aria, etc.) that has been linked to a Fitbit account. Additional tracking from the device indicating whether  logs were manually input or synced via a digital scale revealed there was low usage even among this subset of 8. Only 2 users could be classified as high, one via manual input, the other through automatic input via a linked account to a digital scale. 

Among users of both manual or auto-input methods, we see a low number of records suggesting that regardless of input method, users did not feel the need to continue using this functionality. This points to possible lack of perceived benefit from the information presented by the app concerning weight.


## **User Habits**

In this next section, we'll take a closer look at the activity and sleep habits by this group of users during the 1-month period. 

### Daily Activity Level

Let's first look at the activity level of users over the course of the week. We'll use the total number of steps (`TotalSteps`) recorded daily as a more intuitive measure of physical activity. 

Steps are calculated by the Fitbit device taking the distance moved divided by the stride length (this is determined by using height measurements provided by the user). 

While the Fitbit also calculates `Calories` burned, this is a less direct method to measure activity since the metric incorporates additional variables: heart rate (a faster rate will burn more calories) and the Basal Metabolic Rate (BMR) which estimates how many calories are burned at rest (estimated by Fitbit using user-provided height, gender, weight, age).

In [85]:
%%bigquery
WITH individual_avg AS (    
    SELECT Id,
           AVG(TotalSteps) AS individual_avg_steps
      FROM `gac-bellabeat.activity.daily`
     GROUP BY 1
) 
SELECT ROUND(AVG(ia.individual_avg_steps), 0) AS group_avg_steps
  FROM individual_avg ia


Downloading: 100%|██████████| 1/1 [00:01<00:00,  1.42s/rows]


,group_avg_steps
0,7519.0


In [82]:
%%bigquery
WITH individual_avg AS ( 
  SELECT Id,
         FORMAT_DATE('%a', ActivityDate) AS DayOfWeek,
         FORMAT_DATE('%w', ActivityDate) AS DayOfWeekOrder,
         AVG(TotalSteps) AS AvgByDayOfWeek
    FROM `gac-bellabeat.activity.daily`
   GROUP BY 1, 2, 3
   ORDER BY 2 ASC
)
SELECT ia.DayOfWeek,
       ROUND(AVG(ia.AvgByDayOfWeek), 0) AS cohort_avg_total_steps
  FROM individual_avg ia
 GROUP BY 1, ia.DayOfWeekOrder
 ORDER BY ia.DayOfWeekOrder


Downloading: 100%|██████████| 7/7 [00:01<00:00,  4.94rows/s]


,DayOfWeek,cohort_avg_total_steps
0,Sun,6950.0
1,Mon,7785.0
2,Tue,7993.0
3,Wed,7512.0
4,Thu,7202.0
5,Fri,7328.0
6,Sat,8303.0


In [105]:
%%bigquery

  SELECT 
         FORMAT_DATE('%a', ActivityDate) AS DayOfWeek,
         FORMAT_DATE('%w', ActivityDate) AS DayOfWeekOrder,
         AVG(TotalSteps) AS AvgByDayOfWeek
    FROM `gac-bellabeat.activity.daily`
   GROUP BY 1, 2
   ORDER BY 2


Downloading: 100%|██████████| 7/7 [00:01<00:00,  4.92rows/s]


,DayOfWeek,DayOfWeekOrder,AvgByDayOfWeek
0,Sun,0,6933.231405
1,Mon,1,7780.866667
2,Tue,2,8125.006579
3,Wed,3,7559.373333
4,Thu,4,7405.836735
5,Fri,5,7448.230159
6,Sat,6,8152.975806


In [90]:
%%bigquery
SELECT Id,
       FORMAT_DATE('%a', ActivityDate) AS DayOfWeek,
       FORMAT_DATE('%w', ActivityDate) AS DayOfWeekOrder,
       AVG(TotalSteps) AS AvgByDayOfWeek
FROM `gac-bellabeat.activity.daily`
GROUP BY 1, 2, 3
ORDER BY 2 ASC

Downloading: 100%|██████████| 228/228 [00:01<00:00, 160.07rows/s]


,Id,DayOfWeek,DayOfWeekOrder,AvgByDayOfWeek
0,3977333714,Fri,5,10060.75
1,4388161847,Fri,5,10826.00
2,5577150313,Fri,5,8533.25
3,7007744171,Fri,5,13024.50
4,7086361926,Fri,5,9670.00
...,...,...,...,...
223,4020332650,Wed,3,2788.20
224,8877689391,Wed,3,19705.40
225,5577150313,Wed,3,5891.20
226,1503960366,Wed,3,12656.60


[INSERT GRAPHIC VIA TABLEAU]

On average, we see that peak activity occurs on Monday, Tuesday, and Saturday, while dips occur on Sunday, Thursday, and Friday. 

We can also get a more refined measure of intensity by examining how long (in minutes) a user exerts oneself. Fitbit has 4 categories measured by number of steps per minute or heart rate: `VeryActiveMinutes`,`FairlyActiveMinutes`, `LightlyActiveMinutes`, `SedentaryMinutes`.


We can also get a more refined measure of intensity by examining how long (in minutes) a user exerts oneself. Fitbit has 4 categories measured by number of steps per minute or heart rate: `VeryActiveMinutes`,`FairlyActiveMinutes`, `LightlyActiveMinutes`, `SedentaryMinutes`.

[Metabolic Equivalent of Task (MET)](https://en.wikipedia.org/wiki/Metabolic_equivalent_of_task)

- Very Active: Activities greater than 6 METs such as jogging, running, cycling at over 10mph, fast swimming
- Fairly Active: Activities between 3 to 6 METs such as yoga, leisurely swimming, cycling under 10mph, brisk walking
- Lightly Active: Activities between 1.5 to 3 METs such as walking at a slow pace
- Sedentary: Activities less than 1.5 METs: sitting quietly


Sedentary, Light, Fairly, and Very Active, minutes can be defined as the following metabolic equivalent of tasks (METs) for each level of activity: (Source https://en.wikipedia.org/wiki/Metabolic_equivalent_of_task)
Sedentary: Activities less than 1.5 METs
Light: Activities less than 3 METs
Fairly/Moderate: Activities between 3 - 6 METs
Very: Activities greater than 6 METs


Moderate-intensity exercises include brisk walking, easy jogging, elliptical trainer, leisurely swimming, water aerobics, cycling at less than 10 mph, ballroom or line dancing, and gardening. Vigorous-intensity exercises include running, walking uphill, cycling at over 10 mph, swimming fast, lap swimming, fast or aerobic dancing, sports with lots of running (such as soccer, hockey, basketball, singles tennis), and heavy gardening.



In [159]:
%%bigquery
WITH indiv_daily_avg AS (
    SELECT Id,
           FORMAT_DATE('%a', ActivityDate) AS DayOfWeek,
           FORMAT_DATE('%w', ActivityDate) AS DayOfWeekOrder,
           AVG(VeryActiveMinutes) AS AvgVeryActiveMinutes,
           AVG(FairlyActiveMinutes) AS AvgFairlyActiveMinutes,
           AVG(LightlyActiveMinutes) AS AvgLightlyActiveMinutes,
           AVG(SedentaryMinutes) AS AvgSedentaryMinutes    
    FROM `gac-bellabeat.activity.daily`
    GROUP BY 1, 2, 3
)
SELECT DayOfWeek,
       DayOfWeekOrder,
       ROUND(AVG(AvgVeryActiveMinutes), 1) AS GroupAvgVeryActiveMinutes,
       ROUND(AVG(AvgFairlyActiveMinutes), 1) AS GroupAvgFairlyActiveMinutes,
       ROUND(AVG(AvgLightlyActiveMinutes), 1) AS GroupAvgLightlyActiveMinutes,
       ROUND(AVG(AvgSedentaryMinutes), 1) AS GroupAvgSedentaryMinutes
  FROM indiv_daily_avg
 GROUP BY 1, 2
 ORDER BY 2
    

Downloading: 100%|██████████| 7/7 [00:01<00:00,  4.06rows/s]


,DayOfWeek,DayOfWeekOrder,GroupAvgVeryActiveMinutes,GroupAvgFairlyActiveMinutes,GroupAvgLightlyActiveMinutes,GroupAvgSedentaryMinutes
0,Sun,0,19.7,14.1,175.8,991.9
1,Mon,1,22.8,14.2,194.7,1028.9
2,Tue,2,21.9,13.5,198.7,1018.6
3,Wed,3,20.1,12.8,191.0,1000.4
4,Thu,4,18.7,12.1,181.9,979.8
5,Fri,5,19.3,11.9,201.5,996.9
6,Sat,6,22.2,15.8,208.9,965.1


In [103]:
%%bigquery
WITH indiv_daily_avg AS (
    SELECT Id,
           FORMAT_DATE('%a', ActivityDate) AS DayOfWeek,
           FORMAT_DATE('%w', ActivityDate) AS DayOfWeekOrder,
           ROUND(AVG(VeryActiveMinutes), 0) AS AvgVeryActiveMinutes,
           ROUND(AVG(FairlyActiveMinutes), 0) AS AvgFairlyActiveMinutes,
           ROUND(AVG(LightlyActiveMinutes), 0) AS AvgLightlyActiveMinutes,
           ROUND(AVG(SedentaryMinutes), 0) AS AvgSedentaryMinutes    
    FROM `gac-bellabeat.activity.daily`
    GROUP BY 1, 2, 3
),
daily_group_avg AS (
    SELECT DayOfWeek,
           DayOfWeekOrder,
           ROUND(AVG(AvgVeryActiveMinutes), 1) AS GroupAvgVeryActiveMinutes,
           ROUND(AVG(AvgFairlyActiveMinutes), 1) AS GroupAvgFairlyActiveMinutes,
           ROUND(AVG(AvgLightlyActiveMinutes), 1) AS GroupAvgLightlyActiveMinutes,
           ROUND(AVG(AvgSedentaryMinutes), 1) AS GroupAvgSedentaryMinutes
      FROM indiv_daily_avg
     GROUP BY 1, 2
)
SELECT ROUND(AVG(GroupAvgVeryActiveMinutes), 1) AS WeeklyGroupAvgVeryActiveMinutes,
       ROUND(AVG(GroupAvgFairlyActiveMinutes), 1) AS WeeklyGroupAvgFairlyActiveMinutes,
       ROUND(AVG(GroupAvgLightlyActiveMinutes), 1) AS WeeklyGroupAvgLightlyActiveMinutes,
       ROUND(AVG(GroupAvgSedentaryMinutes), 1) AS WeeklyGroupAvgSedentaryMinutes
  FROM daily_group_avg


Downloading: 100%|██████████| 1/1 [00:01<00:00,  1.41s/rows]


,WeeklyGroupAvgVeryActiveMinutes,WeeklyGroupAvgFairlyActiveMinutes,WeeklyGroupAvgLightlyActiveMinutes,WeeklyGroupAvgSedentaryMinutes
0,20.7,13.5,193.3,997.5


[INSERT GRAPHIC VIA TABLEAU]

By breaking out the activity metric by levels of intensity, we get a more refined picture of usage.

*Monday, Tuesday*
At the beginning of the work week we see peaks in both high intensity `VeryActiveMinutes` and inactivity `SedentaryMinutes`. Even among the middle categories of intensity `FairlyActiveMinutes` and `LightlyActiveMinutes`, we see users staying moderately active. On Tuesday, more users become more active as `SedentaryMinutes` trend down, while we see a slight increase in both levels of moderate intensity, and the most active `VeryActiveMinutes` staying high.    

*Wednesday*
Midweek we see a large decrease at the higher intensities (`VeryActiveMinutes`,`FairlyActiveMinuetes`), and a lesser decrease with `LightlyActiveMinutes`and `SedentaryMinutes`.

*Thursday, Friday*
Heading into the end of the work week, Thursday shows lows relative lows across all intensities with only a slight decrease in `LightlyActiveMinutes`suggesting that most users have shifted to lighter intensity activities like walking. On Friday, there is still a trend to avoid moderate or high intensity activities indicated by the relatively low`VeryActiveMinutes` and `FairlyActiveMinutes`. However, engaging in low-intensity activities move towards a weekly peak as well as some users deciding to rest as indicated by a relative peak in `SedentaryMinutes`.

*Saturday, Sunday*
At the beginning of the weekend, we see high levels of activity at all active intensities (`VeryActiveMinutes`, `FairlyActiveMinuetes`, `LightlyActiveMinutes`) and a accompanying drop in `SedentaryMinutes`. As the weekend draws to a close on Sunday, users are engaging in moderately intense activites (`FairlyActiveMinutes`) or resting (`SedentaryMinutes`).




There is a relative higher level of `LightlyActiveMinutes` on Thursday and Friday perhaps reflecting individuals are recovering from more strenuous workouts earlier in the week and opting instead for lower intensity exercise. 

Interestingly, the only intensity value to have relatively high levels on Sunday is `FairlyActiveMinutes` suggesting that people may still want to remain active on this day, but do not push to the extreme.  

Takeaway: 
- high-intensity activities are happening at the beginning of the work week (Mon, Tues) and weekend (Sat)
- End of the week tends to be lighter intensity activity
- Rest occurs in the beginning part of the week (Sun-Wed) and (Friday)

Marketing Strategy--- 

TABLEAU [Possible graphic] stacked line graph.

### Level of Intensity throughout the Day

Let's now examine what activity levels are like throughout the day. 

In [167]:
%%bigquery
WITH indiv_hourly_avg AS (
    SELECT Id,
           EXTRACT(hour FROM ActivityHour) AS HourOfDay,
           ROUND(AVG(StepTotal), 0) AS AvgStepTotal
     FROM `gac-bellabeat.activity.hourly`
    GROUP BY 1, 2
)
SELECT HourOfDay,
       ROUND(AVG(AvgStepTotal), 0) AS GroupAvgSteps
 FROM indiv_hourly_avg
GROUP BY 1
ORDER BY 1

Downloading: 100%|██████████| 24/24 [00:01<00:00, 16.88rows/s]


,HourOfDay,GroupAvgSteps
0,0,40.0
1,1,22.0
2,2,16.0
3,3,7.0
4,4,12.0
5,5,41.0
6,6,172.0
7,7,318.0
8,8,453.0
9,9,433.0


In [168]:
%%bigquery
WITH indiv_hourly_avg AS (
    SELECT Id,
           FORMAT_DATE('%a', ActivityHour) AS DayOfWeek,
           FORMAT_DATE('%w', ActivityHour) AS DayOfWeekOrder,
           EXTRACT(hour FROM ActivityHour) AS HourOfDay,
           ROUND(AVG(StepTotal), 0) AS AvgStepTotal
     FROM `gac-bellabeat.activity.hourly`
    GROUP BY 1, 2, 3, 4
)
SELECT DayOfWeek,
       DayOfWeekOrder,
       HourOfDay,
       ROUND(AVG(AvgStepTotal), 0) AS GroupAvgSteps
 FROM indiv_hourly_avg
GROUP BY 1, 2, 3
ORDER BY 2, 3

Downloading: 100%|██████████| 168/168 [00:01<00:00, 118.21rows/s]


,DayOfWeek,DayOfWeekOrder,HourOfDay,GroupAvgSteps
0,Sun,0,0,79.0
1,Sun,0,1,50.0
2,Sun,0,2,47.0
3,Sun,0,3,5.0
4,Sun,0,4,6.0
...,...,...,...,...
163,Sat,6,19,571.0
164,Sat,6,20,406.0
165,Sat,6,21,342.0
166,Sat,6,22,307.0


## **Habits** | Sleep

In [126]:
%%bigquery
SELECT *
FROM `gac-bellabeat.activity.sleep`
LIMIT 10

Downloading: 100%|██████████| 10/10 [00:01<00:00,  7.04rows/s]


,Id,SleepDay,TotalSleepRecords,TotalMinutesAsleep,TotalTimeInBed
0,1503960366,2016-04-12 00:00:00+00:00,1,327,346
1,1503960366,2016-04-15 00:00:00+00:00,1,412,442
2,1503960366,2016-04-17 00:00:00+00:00,1,700,712
3,1503960366,2016-04-19 00:00:00+00:00,1,304,320
4,1503960366,2016-04-20 00:00:00+00:00,1,360,377
5,1503960366,2016-04-21 00:00:00+00:00,1,325,364
6,1503960366,2016-04-23 00:00:00+00:00,1,361,384
7,1503960366,2016-04-24 00:00:00+00:00,1,430,449
8,1503960366,2016-04-25 00:00:00+00:00,1,277,323
9,1503960366,2016-04-26 00:00:00+00:00,1,245,274


In [141]:
%%bigquery
WITH indiv_sleep_daily_avg AS (
    SELECT Id,
           FORMAT_DATE('%a', SleepDay) AS DayOfWeek,
           FORMAT_DATE('%w', SleepDay) AS DayOfWeekOrder,
           ROUND(AVG(TotalMinutesAsleep), 0) AS AvgSleepMin,
           ROUND(AVG(TotalMinutesAsleep)/60, 2) AS AvgSleepHour
    FROM `gac-bellabeat.activity.sleep`
    GROUP BY 1, 2, 3 
)
SELECT isda.DayOfWeek,
       isda.DayOfWeekOrder,
       ROUND(AVG(AvgSleepMin), 0) AS GroupAvgSleepMin,
       ROUND(AVG(AvgSleepHour), 2) AS GroupAvgSleepHour
 FROM indiv_sleep_daily_avg isda
GROUP BY 1, 2
ORDER BY 2;

Downloading: 100%|██████████| 7/7 [00:01<00:00,  4.94rows/s]


,DayOfWeek,DayOfWeekOrder,GroupAvgSleepMin,GroupAvgSleepHour
0,Sun,0,425.0,7.08
1,Mon,1,436.0,7.26
2,Tue,2,404.0,6.73
3,Wed,3,429.0,7.14
4,Thu,4,373.0,6.22
5,Fri,5,394.0,6.57
6,Sat,6,376.0,6.27


In [143]:
%%bigquery
WITH indiv_bed_daily_avg AS (
    SELECT Id,
           FORMAT_DATE('%a', SleepDay) AS DayOfWeek,
           FORMAT_DATE('%w', SleepDay) AS DayOfWeekOrder,
           ROUND(AVG(TotalTimeInBed), 0) AS AvgTimeInBedMin,
           ROUND(AVG(TotalTimeInBed)/60, 2) AS AvgTimeInBedHour
     FROM `gac-bellabeat.activity.sleep`
    GROUP BY 1, 2, 3 
)
SELECT ibda.DayOfWeek,
       ibda.DayOfWeekOrder,
       ROUND(AVG(AvgTimeInBedMin), 0) AS GroupAvgInBedMin, 
       ROUND(AVG(AvgTimeInBedHour), 2) AS GroupAvgInBedHour
 FROM indiv_bed_daily_avg ibda
GROUP BY 1, 2
ORDER BY 2


Downloading: 100%|██████████| 7/7 [00:01<00:00,  5.02rows/s]


,DayOfWeek,DayOfWeekOrder,GroupAvgInBedMin,GroupAvgInBedHour
0,Sun,0,480.0,8.00
1,Mon,1,478.0,7.97
2,Tue,2,440.0,7.33
3,Wed,3,464.0,7.73
4,Thu,4,404.0,6.73
5,Fri,5,438.0,7.30
6,Sat,6,416.0,6.93


In [158]:
%%bigquery
WITH indiv_sleep_daily_avg AS (
    SELECT Id,
           FORMAT_DATE('%a', SleepDay) AS DayOfWeek,
           FORMAT_DATE('%w', SleepDay) AS DayOfWeekOrder,
           ROUND(AVG(TotalMinutesAsleep), 0) AS AvgSleepMin,
           ROUND(AVG(TotalMinutesAsleep)/60, 2) AS AvgSleepHour
    FROM `gac-bellabeat.activity.sleep`
    GROUP BY 1, 2, 3 
), 
group_sleep_daily_avg AS (
    SELECT isda.DayOfWeek,
           isda.DayOfWeekOrder,
           ROUND(AVG(AvgSleepMin), 0) AS GroupAvgSleepMin,
           ROUND(AVG(AvgSleepHour), 2) AS GroupAvgSleepHour
     FROM indiv_sleep_daily_avg isda
    GROUP BY 1, 2
),
indiv_bed_daily_avg AS (
    SELECT Id,
           FORMAT_DATE('%a', SleepDay) AS DayOfWeek,
           FORMAT_DATE('%w', SleepDay) AS DayOfWeekOrder,
           ROUND(AVG(TotalTimeInBed), 0) AS AvgTimeInBedMin,
           ROUND(AVG(TotalTimeInBed)/60, 2) AS AvgTimeInBedHour
     FROM `gac-bellabeat.activity.sleep`
    GROUP BY 1, 2, 3 
),
group_bed_daily_avg AS (
    SELECT ibda.DayOfWeek,
           ibda.DayOfWeekOrder,
           ROUND(AVG(AvgTimeInBedMin), 0) AS GroupAvgInBedMin, 
           ROUND(AVG(AvgTimeInBedHour), 2) AS GroupAvgInBedHour
     FROM indiv_bed_daily_avg ibda
    GROUP BY 1, 2
)
SELECT gsda.DayOfWeek,
       gsda.DayofWeekOrder,
       gbda.GroupAvgInBedMin - gsda.GroupAvgSleepMin AS TimeToFallAsleepMin,
       AVG(gbda.GroupAvgInBedMin - gsda.GroupAvgSleepMin) OVER () AS AvgTimeToFallAsleepMin,
       (gbda.GroupAvgInBedMin - gsda.GroupAvgSleepMin) - AVG(gbda.GroupAvgInBedMin - gsda.GroupAvgSleepMin) OVER () AS DeltaWithAvgMin,
       gbda.GroupAvgInBedHour - gsda.GroupAvgSleepHour AS TimeToFallAsleepHour,
       AVG(gbda.GroupAvgInBedHour - gsda.GroupAvgSleepHour) OVER () AS AvgTimeToFallAsleepHour,
       (gbda.GroupAvgInBedHour - gsda.GroupAvgSleepHour) - AVG(gbda.GroupAvgInBedHour - gsda.GroupAvgSleepHour) OVER () AS DeltaWithAvgHour    
 FROM group_sleep_daily_avg gsda
 JOIN group_bed_daily_avg gbda
   ON gsda.DayOfWeek = gbda.DayOfWeek
ORDER BY 2;



Downloading: 100%|██████████| 7/7 [00:01<00:00,  4.94rows/s]


,DayOfWeek,DayofWeekOrder,TimeToFallAsleepMin,AvgTimeToFallAsleepMin,DeltaWithAvgMin,TimeToFallAsleepHour,AvgTimeToFallAsleepHour,DeltaWithAvgHour
0,Sun,0,55.0,40.428571,14.571429,0.92,0.674286,0.245714
1,Mon,1,42.0,40.428571,1.571429,0.71,0.674286,0.035714
2,Tue,2,36.0,40.428571,-4.428571,0.60,0.674286,-0.074286
3,Wed,3,35.0,40.428571,-5.428571,0.59,0.674286,-0.084286
4,Thu,4,31.0,40.428571,-9.428571,0.51,0.674286,-0.164286
5,Fri,5,44.0,40.428571,3.571429,0.73,0.674286,0.055714
6,Sat,6,40.0,40.428571,-0.428571,0.66,0.674286,-0.014286


## **User Profile**

While there is limited demographic info provided with this dataset, we can use information from the weight log information available. 

In [169]:
%%bigquery
SELECT * 
FROM `gac-bellabeat.activity.weight`
LIMIT 5

Downloading: 100%|██████████| 5/5 [00:01<00:00,  3.62rows/s]


,Id,Date,WeightKg,WeightPounds,Fat,BMI,IsManualReport,LogId
0,4558609924,2016-04-18 23:59:59+00:00,69.699997,153.662190,<NA>,27.250000,True,1461023999000
1,4558609924,2016-04-25 23:59:59+00:00,70.300003,154.984977,<NA>,27.459999,True,1461628799000
2,4558609924,2016-05-01 23:59:59+00:00,69.900002,154.103125,<NA>,27.320000,True,1462147199000
3,4558609924,2016-05-02 23:59:59+00:00,69.199997,152.559879,<NA>,27.040001,True,1462233599000
4,4558609924,2016-05-09 23:59:59+00:00,69.099998,152.339420,<NA>,27.000000,True,1462838399000


In [181]:
%%bigquery
WITH indiv_avg_weight AS (
    SELECT Id,
           ROUND(AVG(WeightPounds), 0) AS AvgWeightLogPounds,
           ROUND(AVG(WeightKg), 0) AS AvgWeightLogKg,
           ROUND(AVG(BMI), 0) AS AvgBMI
    FROM `gac-bellabeat.activity.weight`
    GROUP BY 1
)
SELECT Id,
       AvgWeightLogPounds,
       AVG(AvgWeightLogPounds) OVER() AS GroupAvgWeightLogPounds,
       AvgWeightLogKg,
       AVG(AvgWeightLogKg) OVER() AS GroupAvgWeightLogKg,
       AvgBMI,
       AVG(AvgBMI) OVER() AS GroupAvgBMI,
       ROUND(SQRT(AvgWeightLogKg/AvgBMI), 2) AS HeightM,
       AVG(SQRT(AvgWeightLogKg/AvgBMI)) OVER() AS GroupAvgHeightM,
       ROUND(SQRT(AvgWeightLogKg/AvgBMI)*39.37, 0) AS HeightIn,
       AVG(SQRT(AvgWeightLogKg/AvgBMI)*39.37) OVER() AS GroupAvgHeightIn     
 FROM indiv_avg_weight
ORDER BY 2

Downloading: 100%|██████████| 8/8 [00:01<00:00,  6.11rows/s]


,Id,AvgWeightLogPounds,GroupAvgWeightLogPounds,AvgWeightLogKg,GroupAvgWeightLogKg,AvgBMI,GroupAvgBMI,HeightM,GroupAvgHeightM,HeightIn,GroupAvgHeightIn
0,1503960366,116.0,171.75,53.0,78.0,23.0,28.0,1.52,1.661946,60.0,65.430833
1,2873212765,126.0,171.75,57.0,78.0,22.0,28.0,1.61,1.661946,63.0,65.430833
2,6962181067,136.0,171.75,62.0,78.0,24.0,28.0,1.61,1.661946,63.0,65.430833
3,4558609924,154.0,171.75,70.0,78.0,27.0,28.0,1.61,1.661946,63.0,65.430833
4,4319703577,160.0,171.75,72.0,78.0,27.0,28.0,1.63,1.661946,64.0,65.430833
5,8877689391,188.0,171.75,85.0,78.0,25.0,28.0,1.84,1.661946,73.0,65.430833
6,5577150313,200.0,171.75,91.0,78.0,28.0,28.0,1.80,1.661946,71.0,65.430833
7,1927972279,294.0,171.75,134.0,78.0,48.0,28.0,1.67,1.661946,66.0,65.430833


In [183]:
%%bigquery cdc_classification
WITH indiv_avg_weight AS (
    SELECT Id,
           ROUND(AVG(WeightPounds), 0) AS AvgWeightLogPounds,
           ROUND(AVG(WeightKg), 0) AS AvgWeightLogKg,
           ROUND(AVG(BMI), 0) AS AvgBMI
    FROM `gac-bellabeat.activity.weight`
    GROUP BY 1
)
SELECT Id,
       AvgWeightLogPounds,
       AVG(AvgWeightLogPounds) OVER() AS GroupAvgWeightLogPounds,
       AvgWeightLogKg,
       AVG(AvgWeightLogKg) OVER() AS GroupAvgWeightLogKg,
       AvgBMI,
       CASE 
         WHEN AvgBMI >= 30.0 THEN 'Obese'
         WHEN AvgBMI >= 25.0 THEN 'Overweight'
         WHEN AvgBMI >= 18.5 THEN 'Normal/Healthy'
         ELSE 'Underweight'
       END AS CDCAssessment,     
       AVG(AvgBMI) OVER() AS GroupAvgBMI,
       ROUND(SQRT(AvgWeightLogKg/AvgBMI), 2) AS HeightM,
       AVG(SQRT(AvgWeightLogKg/AvgBMI)) OVER() AS GroupAvgHeightM,
       ROUND(SQRT(AvgWeightLogKg/AvgBMI)*39.37, 0) AS HeightIn,
       AVG(SQRT(AvgWeightLogKg/AvgBMI)*39.37) OVER() AS GroupAvgHeightIn     
 FROM indiv_avg_weight
ORDER BY 2

Downloading: 100%|██████████| 8/8 [00:01<00:00,  4.60rows/s]


In [186]:
path = './data/cdc_classification.csv'
cdc_classification.to_csv(path, index=False)